In [ ]:


import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'calculator:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1432756%2F2371392%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240430%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240430T232831Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3382ccb7d49d316873f83b519d7858bf53d19e56341591a1e8e7e44a0711084e10978cf1e08bb85a8d4119f10fd2fd173995773c6328537ab9dc2bf7196f5872238663169ddd7b9941d366d62f5fa1b5e7e13ce4a3e930e593196f5bd8534b13e5288113c8404c3c509c993c39b5b39b01ceb1b9268d041fff6298a58708377242f6829e57305ae765887dec4898dbcf2ec7c06430d98c807080656e907f88ebbfeca7fdb702731d8a55c3ccee523ae216c96bf1011c89a4d98531b4d7bafd973d9789ed3350781a94e79da274cd26c197b9ecb6b1d8b2f7eb9415337dd2072182f90d35f7fa6b76f979a6e35ca90351a95fbdbed10440e86b5ae43e0eeef163,-testeqn:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1435768%2F2376045%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240430%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240430T232831Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D42e8bc43fcfb6fbb70d07a666fdcef3d014a9fedeea6499a2ab938f92358335ffa656dfc3837b6fa6bf2ebad04c32f55dc878208f480aa5abcf0a3ed75d4503e1b34c6105e2b464508cd510cfa2db312d4d96d55bb1ba229dd00db945dd4530036be638f8ac88d095f349bcc8eed187ad8e3b3a443d3713f76e2aa0c4b3a00199e32b970ac224aeed887c759f5a22d095569d5d8e096d4b311dca25d9afa3f892837ed1ad93b98d4d7166770003adf3711e7db4050a4f207dbf272e0690a89e14064f7305c932d112b2eca9a69234090c5c994d7d590c3dfaec5dc0032f8143a84e1c857de9f34f9322d47ebb0c7e047e242e09ec02152329e88d427ef9a2226,d/sujal013/testeqn:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1435776%2F2376177%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240430%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240430T232832Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7fcc10844d8479d5c18e0e625f166ff5d7b3be9ae79fdb63d08412c4668a9627aca98ff118b7c1b9b151b69178d90647aeb4483c1814ddc879412b541f816c1bd301fd919bd1d11b1b79caecaa1157ba9bd62689b94110d8f30079f268b337e86eb324de8ea8dcba804322fdb624df88d5913eeecf77d58d349ca9695690ca38387a7b175eff25c75bd0fcfcb9828e0e08c19da2c22ea2aee5f22766caa949aac7c512238a0280be1776f1b9b016a8bcc3ce192d26510d41f86b26ad307a8f3259dd80e13644fe95487fb06fe3eee6f432889f1df9668a543a2c4118e7d62147d9d38362b68f579ec65923fe091fe3edd37016bf6847f809c255d3c849b6760d,testinput:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1435886%2F2376220%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240430%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240430T232832Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2979d271df154a74927cf5975ae18e08f16839498840ff7396eae32f2c734c8e011f16b250a05d91efb7d58e1e06fdaa0adf36c12eb58392176d08bc993ff1e9c03910e37125167c28f8623f308f880c96ba7e91dc63ebc25bfedfcd05f5f8e9059ecd7734619a5e5561be099b289096a4790149654d4a47cd2553412473c7d0d87d6baf620c71a8caab794508db70811a9161d675fd277c820c3b0ddf595b144db382980e7d4def2a1f9269e27e78540e0ea7dc8ae3d1913a787d3706a4346ce34173028c423af4c1501d291818804d32ae7af09eaf5e7c985dae2c58ba98d5b3443e4d833b71e3d332db11751f6e1ce02b3ff6067446cafffd59fb0618eee8'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# importing libraries used for the model
!pip install imutils
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from imutils.contours import sort_contours
import imutils

In [ ]:
# using keras for building the architecture of convolutional neural net
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
# loading training and validation data
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    validation_split = 0.25
)

data_path='../input/calculator/All data (Compressed)'
train_set = train_datagen.flow_from_directory(
    data_path,
    target_size = (40, 40),
    color_mode = 'grayscale',
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True,
    subset='training',
    seed = 123
)
valid_set = train_datagen.flow_from_directory(
    data_path,
    target_size = (40, 40),
    color_mode = 'grayscale',
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True,
    subset='validation',
    seed = 123
)

In [ ]:
# model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(40, 40, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(16, activation='softmax'))
adam = tf.keras.optimizers.Adam(lr = 5e-4)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# calculate loss and accuracy on both training and validation set
model.fit(train_set,validation_data=valid_set,epochs=2, verbose=1)

In [ ]:
def prediction(img):
    plt.imshow(img, cmap = 'gray')
    img = cv2.resize(img,(40, 40))
    norm_image = cv2.normalize(img, None, alpha = 0, beta = 1, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    norm_image = norm_image.reshape((norm_image.shape[0], norm_image.shape[1], 1))
    case = np.asarray([norm_image])
    pred = model.predict_classes([case])
    print(pred)

    return ([i for i in train_set.class_indices if train_set.class_indices[i]==(pred[0])][0],pred)

In [ ]:
label_map = (train_set.class_indices)
label_map

In [ ]:
# read the test image
image = cv2.imread('../input/test-data/test.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 30, 150)
# we used contour plotting to recognise a digit or operator in an image
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sort_contours(cnts, method="left-to-right")[0]
# draw a rectangular box around the recognised digit and opertor in the image
chars=[]
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    if w*h>1200:
        roi = gray[y:y + h, x:x + w]
        chars.append(prediction(roi))
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image)

In [ ]:
# chars is list of predicted digit/operator and their corresponding index
chars

In [ ]:
labels=[i for i in train_set.class_indices]
print(labels)

In [ ]:
# a is a list of all the digits and operators that are involved in the expression given, as test image
# b is a list which consist of values of length, of a, at which an operator is found
a = []
b = []
for i in ((chars)):
    if len(a)==0 and i[0][0] in labels[3:]:
        a.append(i[0][0])
    elif len(a)>0 and i[0][0] in labels[4:14]:
        a.append(i[0][0])
    elif len(a)>0 and i[0][0] in labels[:4]:
        a.append(i[0][0])
        b.append(len(a))
    else:
        pass

In [ ]:
# implement the calculator after image recognition
x = 0
for i in b:
    if a[i-1]=='+':
        if ((b.index(i))==0 and len(b)==1):
            x =  (int(''.join(a[:i-1]))) + (int(''.join(a[i:])))
        elif ((b.index(i)) == 0):
            x = (int(''.join(a[:i-1]))) + (int(''.join(a[i : (b[1]-1)])))
        elif (((b.index(i)) > 0) & (b.index(i)) != (len(b)-1)):
            x = x + (int(''.join(a[i : (b[b.index(i)+1]-1)])))
        elif (b.index(i) == (len(b)-1)):
            x = x + (int(''.join(a[i : len(a)])))
    elif a[i-1]=='%':
        if ((b.index(i))==0 and len(b)==1):
            x =  (int(''.join(a[:i-1]))) / (int(''.join(a[i:])))
        elif ((b.index(i)) == 0):
            x = (int(''.join(a[:i-1]))) / (int(''.join(a[i : (b[1]-1)])))
        elif (((b.index(i)) > 0) & (b.index(i)) != (len(b)-1)):
            x = x  / (int(''.join(a[i : (b[b.index(i)+1]-1)])))
        elif (b.index(i) == (len(b)-1)):
            x = x  / (int(''.join(a[i : len(a)])))
    elif a[i-1]=='*':
        if ((b.index(i))==0 and len(b)==1):
            x =  (int(''.join(a[:i-1]))) * (int(''.join(a[i:])))
        elif ((b.index(i)) == 0):
            x = (int(''.join(a[:i-1]))) * (int(''.join(a[i : (b[1]-1)])))
        elif (((b.index(i)) > 0) & (b.index(i)) != (len(b)-1)):
            x = x  * (int(''.join(a[i : (b[b.index(i)+1]-1)])))
        elif (b.index(i) == (len(b)-1)):
            x = x  * (int(''.join(a[i :])))
    else:
        if ((b.index(i))==0 and len(b)==1):
            x =  (int(''.join(a[:i-1]))) - (int(''.join(a[i:])))
        elif ((b.index(i)) == 0):
            x = (int(''.join(a[:i-1]))) - (int(''.join(a[i : (b[1]-1)])))
        elif (((b.index(i)) > 0) & (b.index(i)) != (len(b)-1)):
            x = x  - (int(''.join(a[i : (b[b.index(i)+1]-1)])))
        elif (b.index(i) == (len(b)-1)):
            x = x  - (int(''.join(a[i : len(a)])))
print('The result of given operation is - ')
print(x)